In [1]:
from pathlib import Path
from omnibelt import toposort, load_json, save_json, pformat, pformat_vars
from itertools import product, combinations
# import pyperclip
import random
from PIL import Image
from graphviz import Digraph
import matplotlib.pyplot as plt
import io
# import clipboard
import networkx as nx
from tqdm.notebook import tqdm
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns
from omniply import tool, ToolKit, Context

import numpy as np
import torch
from causalbenchmark.novo.simulation.models import Bernoulli, ConditionalBernoulli, BernoulliNetwork
from causalbenchmark.novo.seeding import NewsHeadlines, StatisticsPrompting, Story
# from causalbenchmark.novo.seeding.prompting import GraphInfo
from causalbenchmark.novo.templating import FixedTemplate, SimpleTemplater
# from causalbenchmark.novo.verbalization import misc
from causalbenchmark.novo import misc
from causalbenchmark.novo.verbalization.decision import Decision
from causalbenchmark.novo.templating import Template
from causalbenchmark.novo.verbalization import Verbalizer, MarginalVerbalization, ConditionalVerbalization

In [2]:
use_old_loading = False
story_names = [
	# 'old/test19', 'old/test20', 'old/test21',
	
	# 'Facebook-WallStreet_Trends_Post-Verdict1',
	# 'RoyalFashionTrends_Analysis2',
	'EU Gas Alliance and Inflation Trends',
]
stories = []
for name in story_names:
	story = Story(story_root=misc.data_root() / 'stories', story_id=name)#.populate_defaults()
	stories.append(story)
	if use_old_loading:
		story['stats'] = {int(k): v for k, v in story['stats'].items()}
		story['verbs'] = {k: {int(k2): v2 for k2, v2 in v.items()} for k, v in story['verbs'].items()}
		for k, v in story['verbs'].items():
			for k2, v2 in v.items():
				if 'value' in v2:
					v2['value'] = {int(k3): v3 for k3, v3 in v2['value'].items()}
		story['questions']['questions'] = {int(k): v for k, v in story['questions']['questions'].items()}
len(stories)

In [3]:
def old_story_atoms(story):
	varverbs = {}
	for var, raw in story['verbs'].items():
		info = {'values': {0: {}, 1: {}}}
		
		info['variable'] = raw[1]['variable']
		info['subject'] = raw[2]['subject']
		info['domain'] = raw[4]['domain']
		
		for i in [0, 1]:
			info_val = info['values'][i]
			info_val['verb'] = raw[2]['value'][i] # goes with subject
			info_val['phrase'] = raw[3]['value'][i]
			info_val['descriptor'] = raw[4]['value'][i] # goes with domain
			info_val['event'] = raw[5]['value'][i]
			info_val['conditional'] = raw[6]['value'][i]
			info_val['interventional'] = raw[7]['value'][i]
			
		varverbs[var] = info
	
	story['sheet'] = varverbs
	return story

def story_atoms(story):
	base_keys = ['descriptor', 'subject', 'pronoun', 'preposition', 'domain']
	value_keys = ['predicate', 'nounclause', 'subclause', 'condition', 'action']
	for node in story['nodes']:
		raw = story['verbs'][node['name']]
		info = {'values': {0: {}, 1: {}}}
		info.update({key: raw[key] for key in base_keys})
		for i in [0, 1]:
			info_val = info['values'][i]
			info_val.update({key: raw[f'{key}{i}'] for key in value_keys})
		node['verbs'] = info
for story in stories:
	(old_story_atoms if use_old_loading else story_atoms)(story)

In [4]:
story = random.choice(stories)
# rawverbs = story['verbs']
# rawvars = {node['name']: node for node in story['nodes']}
# next(iter(rawverbs.values()))

In [5]:
node = random.choice(list(story['nodes']))
varval = 0
node['name'], node['values'][varval]

('EU Gas Alliance Formation', 'No Alliance')

In [6]:
atoms = {**node['verbs'], **node['verbs']['values'][varval]}
del atoms['values']
atoms.keys()

dict_keys(['descriptor', 'subject', 'pronoun', 'preposition', 'domain', 'predicate', 'nounclause', 'subclause', 'condition', 'action'])

In [7]:
precise_templates = [
	['there', 'is', 'a', '{quantity}', '{prob_word}', 'that', '{position}'],
	['the', '{chance_word}', 'that', '{position}', '{"are" if chance_word == "odds" else "is"}', '{quantity}'],
	['{quantity}', 'of', 'the', 'time', '{position}'],
	['{quantity}', 'of', '{domain}', '{descriptor}'],
	['with a', '{confidence_word}', 'of {quantity}, {position}']
] if use_old_loading else [
	['there', 'is', 'a', '{quantity}', '{prob_word}', 'that', '{position}'],
	['the', '{chance_word}', 'that', '{position}', '{"are" if chance_word == "odds" else "is"}', '{quantity}'],
	['{quantity}', 'of', 'the', 'time', '{position}'],
	['{preposition}', '{quantity}', 'of', '{domain},', '{subclause}'],
	['with a', '{confidence_word}', 'of {quantity}, {position}']
]
position_templates = [
	['{subject}', '{verb}',],
	['{domain}', '{descriptor}'],
	# ['{descriptor}'],
	['{phrase}',],
	['{event}',],
] if use_old_loading else [
	['{subject}', '{predicate}',],
	['{nounclause}',],
	['{subclause}',],
]

cond_templates = [
	['{antecedent}', '{consequent}'],
	['{consequent},', '{antecedent}'],
]
	
def verb_atoms(node, varval=1):
	atoms = {**node['verbs'], **node['verbs']['values'][varval]}
	del atoms['values']
	return atoms
	
def sample_text(ident=None, **static):
	if ident is None:
		ident = {}
	story_id = ident.setdefault('story_id', random.randint(0,len(stories)-1))
	story = stories[story_id]
	
	var_id = ident.setdefault('var_id', random.randint(0,len(story['nodes'])-1))
	node = story['nodes'][var_id]
	varname = node['name']
	varval = ident.setdefault('value_id', random.choice([0, 1]))
	base_atoms = verb_atoms(node, varval)
	
	conditions = None
	
	parents = [val for key, val in sorted(ident.items()) if key.startswith('parent_id')]
	if len(parents) or ident.setdefault('no_parents', random.choice([False, True])):
		parent_nodes = [node for i, node in enumerate(story['nodes']) if i != var_id]
		if not len(parents):
			num = ident.setdefault('num_parents', random.randint(1, 2))
			parents = random.sample([i for i in range(len(story['nodes'])) if i != var_id], k=num)
		
		pvals = ident.setdefault('parent_vals', [random.randint(0, 1) for _ in range(len(parents))])
		conditions = [verb_atoms(node, pval) for node, pval in zip(parent_nodes, pvals)]
		for i in parents:
			ident[f'parent_id{i}'] = i
	
	V = MarginalVerbalization() if conditions is None else ConditionalVerbalization()
	default_pops = dict(position_templates=position_templates, precise_templates=precise_templates)
	if conditions is not None:
		default_pops['cond_templates'] = cond_templates
	V.populate_default(**default_pops)
	ctx = Verbalizer().populate_variable_info(base_atoms, conditions, verbalization=V, **static)#.populate_default(precise_templates=precise_templates, position_templates=position_templates)
	
	ctx.update(ident)
	return ctx

In [8]:
samples = []


prior = {
	'story_id': 0,
	'var_id': 0,
	'value_id': 1,
	
	# 'clause_id': 0,
	# 'position_id': 3,
	
	'parent_id1': 0,
	'parent_vals': [1],
	# 'parent_id2': 1,
	
}

for _ in range(30):
	ident = prior.copy()
	ctx = sample_text(ident, mean=0.38)
	text = ctx['sentence']
	ident.update(ctx.identity())
	ctx['ident'] = ident
	samples.append(ctx)

cols = []
for row in samples:
	for key in row['ident']:
		if key not in cols:
			cols.append(key)

table = [(i, len([k for k in row['ident'] if k not in prior]),
		  row['sentence'], 
		  # row['ctx']['event'], row['ctx']['event'], row['ctx']['phrase'], row['ctx']['subject'], row['ctx']['verb'],  
		  # *[row['ident'].get(key, '-') for key in cols]
		  ) for i, row in enumerate(samples)]
print(tabulate(table, headers=['i', 'dof', 'text', *cols]))# *list(map(str, range(len(cols))))]))

  i    dof  text
---  -----  --------------------------------------------------------------------------------------------------------
  0      6  38% of the time global energy prices are fluctuating, with an unstable EU economy.
  1      6  38% of the time global energy prices are fluctuating, with an unstable EU economy.
  2      7  With a confidence of 38%, global energy prices are fluctuating, if the EU economy is unstable.
  3      7  The probability that there's volatility in energy markets is 38%, with an unstable EU economy.
  4      6  With an unstable EU economy 38% of the time there is a significant variation in prices.
  5      7  With an unstable EU economy with a certainty of 38%, global energy prices are fluctuating.
  6      7  There is a 38% likelihood that global energy prices are fluctuating, with an unstable EU economy.
  7      7  If the EU economy is unstable with a certainty of 38%, there is a significant variation in prices.
  8      5  Among 38% of energy market

In [78]:
sample = samples[0]
ident = sample['ident']
ctx = sample['ctx']
text = sample['text']

print(text)
print(ident)

The odds that this incident got limited coverage in the news are 62%.
{'story_id': 0, 'var_id': 6, 'value_id': 0, 'verb_id': 2, 'clause_id': 1, 'position_id': 0, 'chance_word_id': 3}


In [ ]:

class ParentVerbalization(ToolKit):
	@tool('antecedent')
	def construct_antecedent(self, parents):
		pass


In [10]:
cond_templates = [
	['{antecedent}', '{clause}'],
	['{clause},', '{antecedent}'],
]

@tool('antecedent')
def construct_antecedent(parents):
	
	
	
	pass


In [ ]:
ident = {}
ctx = sample_text(ident, mean=0.57)



text = Template.detok(ctx['clause'], capitalize=True, sentence=True)
ident.update(ctx.identity())
print(text)
ident

In [33]:

selection = {}




story = random.choice(stories)
varname = random.choice(list(rawverbs.keys()))
varinfo = story['sheet'][varname]
varval = 0

ctx = ClauseVerbalization().populate_variable_info(info, mean=0.57).populate_default(precise_templates=precise_templates, position_templates=position_templates)

# ctx['clause_id'] = 3
ctx['position_id'] = 0

text = Template.detok(ctx['clause'], capitalize=True, sentence=True)
print(text)
ctx.identity()

The chance that this incident was scarcely covered by the media is 57%.


{'verb_id': 1, 'clause_id': 1, 'position_id': 0, 'chance_word_id': 1}

In [10]:
# ClauseVerbalization._precise_templates[0]

In [8]:

prob_words = ['probability', 'chance', 'likelihood']
chance_words = ['probability', 'chance', 'likelihood', 'odds']



In [18]:
def detok(tokens):
	toks = [', ' if tok == ',' else tok for tok in tokens]
	sentence = ' '.join(toks) + '.'
	return sentence.capitalize()

In [ ]:
# 1. One of the variables is {descriptor}.
# 2. {subject} [often/usually/sometimes/rarely/etc.] {predicate}.
# 3. There is a [number]% chance that {noun-clause}.
# 4. [In/For/With/etc.] [number]% of {domain}, there {phrase}.
# 5. Conditional sentence: {condition}, [some consequence].
# 6. Interventional: If {action}, [some effect].

In [19]:
# The man that she loved went home.
# She realized that he loves her.

NameError: name 'AbstractGadget' is not defined

In [ ]:
# 1. One of the variables is {variable}.
# 2. We estimate {subject} [often/usually/sometimes/rarely/etc.] {value}.
# 3. There is a [number]% chance that {value}.
# 4. [number]% of {domain} {value}.
# 5. [number]% of the time {value}.
# 6. Conditional sentence: {value}, [some consequence].
# 7. Interventional: If {value}, [some effect].